## DETECCIÓN DE OUTLIERS

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.covariance import EllipticEnvelope
import matplotlib.pyplot as plt

In [2]:
# Carga de datos.
df = pd.read_csv("outliers.csv")
print(df)

FileNotFoundError: [Errno 2] No such file or directory: 'outliers.csv'

### 1) EllipticEnvelope()

In [ ]:
# Definimos el algoritmo EllipticEnvelope
algorithm = EllipticEnvelope(support_fraction=None, contamination=0.25, random_state=42)

# Entrenamos el algoritmo
outlier_method = algorithm.fit(df)

# Aplicamos el método de detección de outliers entrenado sobre nuesto dataset
df_outliers = outlier_method.predict(df)
print(df_outliers)

# Determinar la posición de los outliers
pos_outliers = np.where(df_outliers==-1)[0]
print('\nOutliers en la posición: \n', pos_outliers)

# Determinar el número de outliers
print('\nNúmero de outliers: \n', len(pos_outliers))

In [ ]:
# Definimos una función que, dado un determinado "df" y un "algorithm", devuelva la matriz y la posición de outliers
def find_outliers(df, algorithm):
    # Entrenamos el algoritmo
    outlier_method = algorithm.fit(df)
    
    # Aplicamos el método de detección de outliers entrenado sobre nuesto dataset
    df_outliers = outlier_method.predict(df)
    #print(df_outliers)

    # Determinar la posición de los outliers
    pos_outliers = np.where(df_outliers==-1)[0]
    print('\nOutliers en la posición: \n', pos_outliers)

    # Determinar el número de outliers
    print('\nNúmero de outliers: \n', len(pos_outliers))
    
    return df_outliers, pos_outliers

### 2) Otros métodos similares

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor

IF = IsolationForest(n_estimators=100, contamination='auto', random_state=42)
OC_SVM =  OneClassSVM(kernel='poly', degree=1, gamma='auto')
LOF = LocalOutlierFactor(n_neighbors=20, algorithm='auto', metric='euclidean', contamination='auto', novelty=True)

df_outliers, pos_outliers = find_outliers(df, IF)

In [ ]:
# Eliminamos los outliers
new_df = df[df_outliers==1]
print(new_df)

### 3) Box plot

In [ ]:
# Seleccionamos el atributo que vamos a medir
a = df['a']

# Seleccionamos los umbrales a partir de los cuales vamos a considerar outliers
Q1 = stats.scoreatpercentile(a, 25)
Q3 = stats.scoreatpercentile(a, 75)
RIC = Q3 - Q1
li = Q1 - 1.5*RIC #xmin
ls = Q3 + 1.5*RIC #xmax

# Observamos los límites inferior y superior
print('limite inferior: ', li)
print('limite superior: ', ls)

# Buscamos la posición de los outliers
pos_i = np.where(a<li)[0]
pos_s = np.where(a>ls)[0]
pos_outliers = np.concatenate((pos_i, pos_s))
print('Posición de outliers: ', pos_outliers)
print('Número de outliers: ', len(pos_outliers))

# Dibujamos el diagrama de caja y bigotes
prop = plt.boxplot(a)
plt.boxplot(a)
plt.grid()
plt.show()

In [ ]:
# Definir una función que, dada una columna de un dataframe, devuelva la posición de los outliers según el método box plot

def find_limits_BP(variable):
    # Seleccionamos los umbrales a partir de los cuales vamos a considerar outliers
    Q1 = stats.scoreatpercentile(variable, 25)
    Q3 = stats.scoreatpercentile(variable, 75)
    RIC = Q3 - Q1
    li = Q1 - 1.5*RIC #xmin
    ls = Q3 + 1.5*RIC #xmax

    # Observamos los límites inferior y superior
    print('limite inferior: ', li)
    print('limite superior: ', ls)

    # Buscamos la posición de los outliers
    pos_i = np.where(variable<li)[0]
    pos_s = np.where(variable>ls)[0]
    pos_outliers = np.concatenate((pos_i, pos_s))
    print('Posición de outliers: ', pos_outliers)
    print('Número de outliers: ', len(pos_outliers))

    # Dibujamos el diagrama de caja y bigotes
    prop = plt.boxplot(variable)
    plt.boxplot(variable)
    plt.grid()
    plt.show()
    
    return pos_outliers

In [ ]:
# Creamos un bucle for que estime los valores outliers de cada atributo
headers = df.columns # nombre de los atributos del CSV
pos_outliers = []
for i in range(len(headers)):
    variable = df[headers[i]] # Atributo i
    pos_out = find_limits_BP(variable) # Buscamos los outliers en esa variable con la función que hemos creado
    pos_out = np.expand_dims(pos_out, axis=1) # Llamamos a la función que hemos creado
    pos_outliers.append(pos_out) # Lo añadimos en una lista

# Concatenamos todas las posiciones de outliers
po = np.vstack(pos_outliers)

# Vemos las posiciones de todos los outliers
pos_out = np.unique(po)
print('Posiciones de outliers totales: ', pos_out)

# Observamos el número de outliers
print('Numero de outliers totales: ', len(pos_out))